In [ ]:
import csv
!pip install stocksymbol
from stocksymbol import StockSymbol

api_key = '74e6f192-55d1-4115-a88b-7e8c17e29b9c'
ss = StockSymbol(api_key)

symbol_list_us = ss.get_symbol_list(market="IN")  # "us" or "america" will also work

file_path = "/content/ticker"

with open(file_path, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Symbol"])
    writer.writerows(zip(symbol_list_us))

print("List of symbols (IN market) saved to symbol_list.csv file.")

In [2]:
import pandas as pd

# Input and output file paths
input_file = 'ticker'
output_file = 'ticker_cleaned'

# Read the CSV file into a DataFrame
df = pd.read_csv(input_file)

# Extract the 'symbol' field and remove other details
df['symbol'] = df['Symbol'].apply(lambda x: eval(x)['symbol'])

# Drop the original 'Symbol' column
df.drop(columns=['Symbol'], inplace=True)

# Write the modified DataFrame back to a new CSV file
df.to_csv(output_file, index=False)

print("Symbols extracted and saved to", output_file)

Symbols extracted and saved to ticker_cleaned


In [ ]:
import yfinance as yf
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

# Input and output file paths
input_file = '/content/ticker_cleaned'
output_file = 'symbol_info.csv'

# Read symbols from the CSV file into a DataFrame
symbols_df = pd.read_csv(input_file)

# Define the list of desired features
desired_features = ['sector','shortName','longName','industry']

# Function to fetch information for a symbol
def fetch_info(symbol):
    try:
        # Use the entered symbol with yf.Ticker() to get the information
        ticker = yf.Ticker(symbol)
        info = ticker.info

        # Create a dictionary to store selected features for the symbol
        selected_info = {'symbol': symbol}

        # Iterate through desired features and retrieve them from info dictionary
        for feature in desired_features:
            if feature in info:
                selected_info[feature] = info[feature]

        return selected_info
    except Exception as e:
        print(f"Error processing symbol {symbol}: {e}")
        return None

# Create an empty DataFrame to store the selected information
selected_info_df = pd.DataFrame(columns=['symbol'] + desired_features)

# Use ThreadPoolExecutor for parallel processing
with ThreadPoolExecutor() as executor:
    # Submit tasks to fetch information for each symbol concurrently
    symbol_info = executor.map(fetch_info, symbols_df['symbol'])

    # Process the results
    for info in symbol_info:
        if info is not None:
            selected_info_df = selected_info_df.append(info, ignore_index=True)

# Save the DataFrame to a CSV file
selected_info_df.to_csv(output_file, index=False)

print("Symbol information extracted and saved to", output_file)


In [5]:
import pandas as pd

# Load the CSV file containing sectors and symbols
df = pd.read_csv('/content/symbol_info.csv')

# Define a function to clean a single symbol
def clean_symbol(symbol):
    return symbol.replace('.NS', '').replace('.BO', '')

# Apply the cleaning function to each symbol in the 'Symbols' column
df['symbol'] = df['symbol'].apply(lambda symbols: ','.join(clean_symbol(symbol) for symbol in symbols.split(',')))

# Save the cleaned DataFrame to a new CSV file
df.to_csv('cleaned_data.csv', index=False)


In [6]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('/content/cleaned_data.csv')

# Remove duplicate rows based on values in the first column
df.drop_duplicates(subset=df.columns[0], keep='first', inplace=True)

# Save the cleaned DataFrame to a new CSV file
df.to_csv('cleaned_data_duplicate.csv', index=False)


In [7]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('/content/cleaned_data_duplicate.csv')

# Sort the DataFrame by 'Sector' column, and within each group, sort by other columns
df_sorted = df.sort_values(by=['sector', 'symbol', 'shortName', 'longName', 'industry'])

# Save the sorted DataFrame to a new CSV file
df_sorted.to_csv('sorted_data.csv', index=False)


In [ ]:
import pandas as pd
import re

# Read the CSV file
df = pd.read_csv('/content/sorted_data.csv')

# Define a regular expression pattern to match specific special symbols
pattern = r'[()@?,&-]'  # Matches any of the specified special symbols

# Remove specific special symbols from all columns
df.replace({pattern: ''}, regex=True, inplace=True)

# Save the modified DataFrame back to CSV
df.to_csv('modified_file.csv', index=False)

print("Specific special symbols removed and saved to modified_file.csv")


In [14]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('/content/modified_file.csv')

# Get the unique values in the 'Sector' column
unique_sectors = df['sector'].unique()

# Iterate over each unique sector
for sector in unique_sectors:
    # Filter the DataFrame for the current sector
    sector_df = df[df['sector'] == sector]

    # Sort the DataFrame by the first column (assuming you want to sort by the first column)
    sector_df_sorted = sector_df.sort_values(by=sector_df.columns[0])

    # Drop the 'Sector' column
    sector_df_sorted.drop(columns=['sector'], inplace=True)

    # Save the sorted DataFrame to a new CSV file
    sector_df_sorted.to_csv(f'{sector}_wisesort.csv', index=False)


In [ ]:
import pandas as pd

# List of CSV files with their names
csv_files = [
    ("/content/Basic Materials_wisesort.csv", "Basic Materials_wisesort.csv"),
    ("/content/Communication Services_wisesort.csv", "Communication Services_wisesort.cs"),
    ("/content/Consumer Cyclical_wisesort.csv", "Consumer Cyclical_wisesort.csv"),
     ("/content/Consumer Defensive_wisesort.csv", "Consumer Defensive_wisesort.csv"),
     ("/content/Energy_wisesort.csv", "Energy_wisesort.csv"),
     ("/content/Financial Services_wisesort.csv", "Financial Services_wisesort.csv"),
     ("/content/Healthcare_wisesort.csv", "Healthcare_wisesort.csv"),
     ("/content/Industrials_wisesort.csv", "Industrials_wisesort.csv"),
     ("/content/Real Estate_wisesort.csv", "Real Estate_wisesort.csv"),
     ("/content/Technology_wisesort.csv", "Technology_wisesort.csv"),
     ("/content/Utilities_wisesort.csv", "Utilities_wisesort.csv"),
     ("/content/nan_wisesort.csv", "nan_wisesort.csv"),
    # Add more CSV files with their names as needed
]

# Iterate over each CSV file
for csv_file, name in csv_files:
    # Create an empty DataFrame to store the individual words
    df_all_words = pd.DataFrame(columns=['All Words'])
    # Read the original CSV file
    df = pd.read_csv(csv_file)
    # Iterate over each cell in the original DataFrame
    for col in df.columns:
        for cell in df[col]:
            # Split the text into words
            words = str(cell).split()
            # Add each word to the new DataFrame
            for word in words:
                df_all_words = df_all_words.append({'All Words': word}, ignore_index=True)

    # Save the DataFrame to a new CSV file with the name of the source CSV file
    output_file = f'all_words_{name}.csv'
    df_all_words.to_csv(output_file, index=False)
    print(f"Individual words extracted and saved to {output_file}")


In [ ]:
import pandas as pd

# List of CSV files
csv_files = [
   "/content/all_words_Basic Materials_wisesort.csv.csv",
    "/content/all_words_Communication Services_wisesort.cs.csv",
    "/content/all_words_Consumer Cyclical_wisesort.csv.csv",
    "/content/all_words_Consumer Defensive_wisesort.csv.csv",
    "/content/all_words_Energy_wisesort.csv.csv",
    "/content/all_words_Financial Services_wisesort.csv.csv",
    "/content/all_words_Healthcare_wisesort.csv.csv",
    "/content/all_words_Industrials_wisesort.csv.csv",
    "/content/all_words_Real Estate_wisesort.csv.csv",
    "/content/all_words_Technology_wisesort.csv.csv",
    "/content/all_words_Utilities_wisesort.csv.csv",
    "/content/all_words_nan_wisesort.csv.csv"
    # Add more CSV files as needed
]

# Iterate over each CSV file
for csv_file in csv_files:
    # Set to store unique words for each file
    unique_words = set()
    # Read the CSV file
    df = pd.read_csv(csv_file)
    # Iterate over each row in the DataFrame
    for row in df.itertuples(index=False):
        # Split the text into words and add them to the set
        words = str(row[0]).split()  # Assuming the text is in the first column
        unique_words.update(words)
    # Convert the set of unique words to a DataFrame
    df_unique_words = pd.DataFrame(unique_words, columns=['Unique Words'])
    # Save the DataFrame to a new CSV file
    output_file = f"unique_words_{csv_file.split('/')[-1]}"
    df_unique_words.to_csv(output_file, index=False)
    print(f"Unique words extracted and saved to {output_file}")


In [20]:
import pandas as pd
import os

# List of CSV files containing unique words
csv_files = [
    "/content/unique_words_all_words_Basic Materials_wisesort.csv.csv",
    "/content/unique_words_all_words_Communication Services_wisesort.cs.csv",
    "/content/unique_words_all_words_Consumer Cyclical_wisesort.csv.csv",
    "/content/unique_words_all_words_Consumer Defensive_wisesort.csv.csv",
    "/content/unique_words_all_words_Energy_wisesort.csv.csv",
    "/content/unique_words_all_words_Financial Services_wisesort.csv.csv",
    "/content/unique_words_all_words_Healthcare_wisesort.csv.csv",
    "/content/unique_words_all_words_Industrials_wisesort.csv.csv",
    "/content/unique_words_all_words_Real Estate_wisesort.csv.csv",
    "/content/unique_words_all_words_Technology_wisesort.csv.csv",
    "/content/unique_words_all_words_Utilities_wisesort.csv.csv",
    "/content/unique_words_all_words_nan_wisesort.csv.csv"
    # Add more CSV files as needed
]

# Read the headlines CSV file
headlines_df = pd.read_csv("/content/cleaned_combined.csv")

# Iterate over each CSV file containing unique words
for csv_file in csv_files:
    # Read the CSV file containing unique words
    unique_words_df = pd.read_csv(csv_file)

    # Initialize an empty DataFrame to store filtered headlines
    filtered_headlines = pd.DataFrame(columns=headlines_df.columns)

    # Get the filename (without extension) for the output CSV file
    output_filename = os.path.splitext(os.path.basename(csv_file))[0] + "_filtered_headlines.csv"

    # Iterate over each unique word in the CSV file
    for word in unique_words_df["Unique Words"]:
        # Ensure that the word is a string
        word = str(word)

        # Search for the word in the headlines DataFrame
        filtered_rows = headlines_df[headlines_df["Headline"].str.contains(word, case=False)]

        # Append the filtered rows to the DataFrame
        filtered_headlines = pd.concat([filtered_headlines, filtered_rows])

    # Remove duplicate rows
    filtered_headlines.drop_duplicates(inplace=True)

    # Save the filtered headlines to a new CSV file
    filtered_headlines.to_csv(output_filename, index=False)

    print(f"Filtered headlines for {csv_file} saved to {output_filename}")


Filtered headlines for /content/unique_words_all_words_Basic Materials_wisesort.csv.csv saved to unique_words_all_words_Basic Materials_wisesort.csv_filtered_headlines.csv
Filtered headlines for /content/unique_words_all_words_Communication Services_wisesort.cs.csv saved to unique_words_all_words_Communication Services_wisesort.cs_filtered_headlines.csv
Filtered headlines for /content/unique_words_all_words_Consumer Cyclical_wisesort.csv.csv saved to unique_words_all_words_Consumer Cyclical_wisesort.csv_filtered_headlines.csv
Filtered headlines for /content/unique_words_all_words_Consumer Defensive_wisesort.csv.csv saved to unique_words_all_words_Consumer Defensive_wisesort.csv_filtered_headlines.csv
Filtered headlines for /content/unique_words_all_words_Energy_wisesort.csv.csv saved to unique_words_all_words_Energy_wisesort.csv_filtered_headlines.csv
Filtered headlines for /content/unique_words_all_words_Financial Services_wisesort.csv.csv saved to unique_words_all_words_Financial Ser

In [23]:
import pandas as pd
import os

# List of CSV files containing unique words
csv_files = [
    "/content/unique_words_all_words_Basic Materials_wisesort.csv.csv",
    "/content/unique_words_all_words_Communication Services_wisesort.cs.csv",
    "/content/unique_words_all_words_Consumer Cyclical_wisesort.csv.csv",
    "/content/unique_words_all_words_Consumer Defensive_wisesort.csv.csv",
    "/content/unique_words_all_words_Energy_wisesort.csv.csv",
    "/content/unique_words_all_words_Financial Services_wisesort.csv.csv",
    "/content/unique_words_all_words_Healthcare_wisesort.csv.csv",
    "/content/unique_words_all_words_Industrials_wisesort.csv.csv",
    "/content/unique_words_all_words_Real Estate_wisesort.csv.csv",
    "/content/unique_words_all_words_Technology_wisesort.csv.csv",
    "/content/unique_words_all_words_Utilities_wisesort.csv.csv",
    "/content/unique_words_all_words_nan_wisesort.csv.csv"
    # Add more CSV files as needed
]

# Read the headlines CSV file
headlines_df = pd.read_csv("/content/cleaned_combined.csv")

# Words to be removed from headlines
words_to_remove = ["the", "an", "of", "for", "from", "his", "him"]

# Iterate over each CSV file containing unique words
for csv_file in csv_files:
    # Read the CSV file containing unique words
    unique_words_df = pd.read_csv(csv_file)

    # Initialize an empty DataFrame to store filtered headlines
    filtered_headlines = pd.DataFrame(columns=headlines_df.columns)

    # Get the filename (without extension) for the output CSV file
    output_filename = os.path.splitext(os.path.basename(csv_file))[0] + "_filtered_headlines.csv"

    # Iterate over each unique word in the CSV file
    for word in unique_words_df["Unique Words"]:
        # Ensure that the word is a string
        word = str(word)

        # Check if the word is not in the words_to_remove list
        if word.lower() not in words_to_remove:
            # Search for the word in the headlines DataFrame
            filtered_rows = headlines_df[headlines_df["Headline"].str.contains(word, case=False)]

            # Append the filtered rows to the DataFrame
            filtered_headlines = pd.concat([filtered_headlines, filtered_rows])

    # Remove duplicate rows
    filtered_headlines.drop_duplicates(inplace=True)

    # Save the filtered headlines to a new CSV file
    filtered_headlines.to_csv(output_filename, index=False)

    print(f"Filtered headlines for {csv_file} saved to {output_filename}")


Filtered headlines for /content/unique_words_all_words_Basic Materials_wisesort.csv.csv saved to unique_words_all_words_Basic Materials_wisesort.csv_filtered_headlines.csv
Filtered headlines for /content/unique_words_all_words_Communication Services_wisesort.cs.csv saved to unique_words_all_words_Communication Services_wisesort.cs_filtered_headlines.csv
Filtered headlines for /content/unique_words_all_words_Consumer Cyclical_wisesort.csv.csv saved to unique_words_all_words_Consumer Cyclical_wisesort.csv_filtered_headlines.csv
Filtered headlines for /content/unique_words_all_words_Consumer Defensive_wisesort.csv.csv saved to unique_words_all_words_Consumer Defensive_wisesort.csv_filtered_headlines.csv
Filtered headlines for /content/unique_words_all_words_Energy_wisesort.csv.csv saved to unique_words_all_words_Energy_wisesort.csv_filtered_headlines.csv
Filtered headlines for /content/unique_words_all_words_Financial Services_wisesort.csv.csv saved to unique_words_all_words_Financial Ser